<a href="https://colab.research.google.com/github/emmanuelvaie/google_colab/blob/main/rappro_licences_resa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()
import pandas as pd
import gspread
from google.auth import default
from gspread_dataframe import set_with_dataframe


creds, _ = default()

gc = gspread.authorize(creds) 

In [2]:
worksheet_licence = gc.open_by_url('https://docs.google.com/spreadsheets/d/1cifoZq8-5ILJk5-2R3yZWnX52DohTYoG1cYbeo0Gqss/edit#gid=1576518051').sheet1

In [3]:
licences_2023 = worksheet_licence.spreadsheet.worksheet('2023')

# get_all_values gives a list of rows.
rows_licences = licences_2023.get_all_values()

import pandas as pd
df_licences_2023 = pd.DataFrame.from_records(rows_licences,columns=rows_licences[0])
df_licences_2023 = df_licences_2023.drop(0).reindex()

In [4]:
df_licences_2023['NO ADHERENT'].count()

286

In [5]:
df_licences_2023['NO ADHERENT'] = df_licences_2023['NO ADHERENT'].apply(lambda x: x.strip())
df_licences_2023['NO ADHERENT'] = df_licences_2023['NO ADHERENT'].astype('float')
df_licences_2023['NO ADHERENT'] = df_licences_2023['NO ADHERENT'].astype('Int32')

In [7]:
import datetime
now = datetime.datetime.now()
df_licences_2023['AGE'] = df_licences_2023['DATE NAISSANCE'].apply(lambda x: now.year - pd.to_datetime(x).year)

In [8]:
df_2023 = df_licences_2023[['NO ADHERENT','NOM','PRENOM','EMAIL','AGE']]

In [9]:
df_2023.head()

,NO ADHERENT,NOM,PRENOM,EMAIL,AGE
1,583500,DANTEC,FREDERIC,fdantec@yahoo.fr,0
2,582254,JAUGEAT,NOA,wassila.jaugeat@gmail.com,10
3,579363,HILALI,ZAKARIA,linda.hilali@gmx.fr,10
4,579248,BARALE,ILYAS,vbarale@yaoo.fr,10
5,580524,DE LAMBERTERIE,ADRIEN,guilamberterie@gmail.com,11


In [12]:
resa = worksheet_licence.spreadsheet.worksheet('réservations MC')

# get_all_values gives a list of rows.
rows_resa = resa.get_all_values()

import pandas as pd
df_resa = pd.DataFrame.from_records(rows_resa,columns=rows_resa[0])
df_resa = df_resa.drop(0).reindex()

In [13]:
df_resa.head()

,Date résa,Date session,Statut,Club,Activité,Email,Nom,Prénom,Montant TTC,Mangopay Id
1,2022-10-23,2022-10-29,gratuit inscrit,Masters + Open,aviron,vaiee@hotmail.com,VAIE,Emmanuel,0€,
2,2022-10-23,2022-10-28,gratuit inscrit,J14,aviron,,DEHENNIN,Mattheo,0€,
3,2022-10-23,2022-10-26,gratuit inscrit,J14,aviron,,DEHENNIN,Mattheo,0€,
4,2022-10-23,2022-10-26,gratuit inscrit,Loisirs débutants,aviron,fannyroullet@live.fr,ROULLET,Fanny,0€,
5,2022-10-23,2022-10-24,gratuit inscrit,Loisirs Confirmés,aviron,fannyroullet@live.fr,ROULLET,Fanny,0€,


In [14]:
resa = df_resa[['Nom','Club']]

In [15]:
resa.head()

,Nom,Club
1,VAIE,Masters + Open
2,DEHENNIN,J14
3,DEHENNIN,J14
4,ROULLET,Loisirs débutants
5,ROULLET,Loisirs Confirmés


In [17]:
resa = resa[resa['Club'] != 'Baptêmes - Initiation']

In [18]:
resa = resa[resa['Club'] != 'Stages Jeunes']

In [19]:
resa.count()

Nom     993
Club    993
dtype: int64

In [28]:
df = df_2023.join(resa.set_index('Nom'), on='NOM', lsuffix='Licencie', rsuffix='resa')

In [29]:
df.head()

,NO ADHERENT,NOM,PRENOM,EMAIL,AGE,Club
1,583500,DANTEC,FREDERIC,fdantec@yahoo.fr,0,NaN
2,582254,JAUGEAT,NOA,wassila.jaugeat@gmail.com,10,NaN
3,579363,HILALI,ZAKARIA,linda.hilali@gmx.fr,10,J12
3,579363,HILALI,ZAKARIA,linda.hilali@gmx.fr,10,J12
3,579363,HILALI,ZAKARIA,linda.hilali@gmx.fr,10,J12


In [33]:
pasderesa = df[df['Club'].isna()]

In [34]:
try :
  licences_reprocessed = worksheet_licence.spreadsheet.worksheet('Pas de resa')
  licences_reprocessed.clear()
except:
  licences_reprocessed = worksheet_licence.spreadsheet.add_worksheet(title='Pas resa', rows=200, cols=10)


set_with_dataframe(licences_reprocessed, pasderesa)